In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import layers
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, one_hot
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten, Dropout, Embedding, LSTM
from keras.callbacks import EarlyStopping
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from random import choices
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
word_map = {'ham':0, 'spam':1}

train_data = pd.read_csv(train_file_path, sep='\t', names=['label','text'])
train_data['label'] = train_data['label'].map(word_map)
# train_data['length'] = train_data['text'].apply(lambda x: len(x))
test_data = pd.read_csv(test_file_path, sep='\t', names=['label','text'])
test_data['label'] = test_data['label'].map(word_map)
# test_data['length'] = test_data['text'].apply(lambda x: len(x))
print(train_data.shape, test_data.shape)
train_data.head(),test_data.head()

In [ ]:
# ham_only = train_data[train_data.label==0].sample(frac=0.15)
# print(ham_only.label.value_counts())
# train_balanced_data = pd.concat([ham_only,train_data[train_data.label==1]], ignore_index=True)
# print(train_balanced_data.label.value_counts())
# train_balanced_data.head()

In [ ]:
train_labels = train_data.pop('label')
test_labels = test_data.pop('label')
train_labels.value_counts(), test_labels.value_counts()

In [ ]:
MAXLEN = 30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'].values)
# tokenizer
encoded_train = tokenizer.texts_to_sequences(train_data['text'].values)
encoded_test = tokenizer.texts_to_sequences(test_data['text'].values)
padded_train = pad_sequences(encoded_train, maxlen=MAXLEN, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=MAXLEN, padding='post')
vocab_len = len(tokenizer.word_index) + 1

In [ ]:
# model = Sequential([
#     Embedding(ENCODE_PARAM, 256),
#     LSTM(256),
#     Dense(4, activation='sigmoid'),
#     Dense(1, activation='linear')
# ])
model = Sequential([
    Embedding(35000, 24, input_length=MAXLEN),
    SimpleRNN(24, return_sequences=False),
    Dense(1, activation='sigmoid')
    ])
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=7, verbose=1)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')
history = model.fit(padded_train, train_labels.values, epochs=100, verbose=1, callbacks=[es], validation_data=[padded_test, test_labels])

In [ ]:
model.evaluate(padded_test, test_labels)

In [ ]:
# model.evaluate(padded_train, train_labels)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_encoded = tokenizer.texts_to_sequences([pred_text])
  # print(pred_encoded)
  pred_padded = pad_sequences(pred_encoded, maxlen=MAXLEN, padding='post')
  # print(pred_padded)
  prediction = model.predict(pred_padded).flatten()[0]
  # prediction = model.predict(pred_padded).astype("int32").item()
  # print(prediction)

  if prediction < 0.5:
    return (prediction, 'ham')
  return (prediction, 'spam')
  # return (prediction, 'ham' if prediction < 0.5 else 'spam')

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction, msg)
    if prediction[1] != ans:
      passed = False
      # break

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
